In [22]:
# !pip install nltk
!pip install sentence-transformers

  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
                                              0.0/7.0 MB ? eta -:--:--
                                              0.0/7.0 MB ? eta -:--:--
                                              0.1/7.0 MB 1.1 MB/s eta 0:00:07
     -                                        0.3/7.0 MB 2.6 MB/s eta 0:00:03
     ---                                      0.6/7.0 MB 3.5 MB/s eta 0:00:02
     ------                                   1.1/7.0 MB 5.2 MB/s eta 0:00:02
     --------                                 1.4/7.0 MB 5.4 MB/s eta 0:00:02
     ----------                               1.9/7.0 MB 6.1 MB/s eta 0:00:01
     ----------                               1.9/7.0 MB 6.1 MB/s eta 0:00:01
     ---------------                          2.8/7.0 MB 6.8 MB/s eta 0:00:01
     ------------------                       3.2/7.0 MB 7.0 MB/s eta 0:00

### Get some sample product_ids

In [1]:
import pandas as pd
import numpy as np
import pdb
from tqdm import tqdm
import nltk
import string
import pickle 

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

# Only for the first time run below code
# nltk.download()

In [21]:
metadata_filename = '../Project_Data/Metadata.csv'
metadata = pd.read_csv(metadata_filename, index_col=0)
metadata['description'] = metadata['description'].apply(lambda x: eval(x)[0])
metadata.sample(5)

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin
58915,"['Clothing, Shoes & Jewelry', 'Girls', 'Shoes'...",NaN,Water shoe,"class=""a-normal a-align-center a-spacing-smal...",Teva Scamper Water Shoe (Toddler/Little Kid/Bi...,"['B07DHGZWP3', 'B078GL92V2', 'B0148TL6EC', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,NaN,NaN,"['100% Leather', 'Rubber sole', 'Bungee water ...","256,823 in Clothing, Shoes & Jewelry (","['B078GL92V2', 'B07G9W6LPC', 'B071D4BRWH', 'B0...",{},"<img src=""https://images-na.ssl-images-amazon....",NaN,"<div class=""a-fixed-left-grid a-spacing-none"">...",$15.31 - $99.00,B00YIYV5KI
78108,"['Clothing, Shoes & Jewelry', 'Women', 'Watche...",NaN,Round watch in silver tone featuring crystal-a...,NaN,GUESS Women's U0335L1 Silver-Tone Multi-Functi...,"['B07BFV332M', 'B07BFW8QHZ', 'B07BFV5TXM', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,NaN,NaN,"['Made in USA or Imported', ""Round watch in si...","1,277,178 in Clothing, Shoes & Jewelry (","['B07BFVDN2Q', 'B07BFW8QHZ', 'B07BFW8QL3', 'B0...",{},"<img src=""https://images-na.ssl-images-amazon....",NaN,GUESS,$5.05,B00G3LPNMA
488,"['Clothing, Shoes & Jewelry', 'Costumes & Acce...",NaN,It's HOT! HOT! HOT! The most official looking ...,NaN,"Aeromax Jr. Fire Fighter Bunker Gear, Black, S...","['B0033MIVBW', 'B00HV3B9RW', 'B0744YSYTR', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,NaN,Aeromax,['Durable fabric and design. Great for imagina...,"['>#187,806 in Toys & Games (See Top 100 in To...","['B01782B074', 'B0033MIVBW', 'B01782BA60', 'B0...",{},Toys & Games,NaN,NaN,$44.86,B00007KQBV
81000,"['Clothing, Shoes & Jewelry', 'Novelty & More'...",NaN,Officially licensed Jack Daniels Classic Label...,"class=""a-normal a-align-center a-spacing-smal...",Jack Daniel's Long-sleeved Label T-shirt,"['B07JN9KB15', 'B07CQKKKHQ', 'B005EVCP1C', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,NaN,Jack Daniels,"['100% Cotton', 'High Quality Item; Officially...","276,976 in Clothing, Shoes & Jewelry (","['B07JQ4RL55', 'B07C6DR6BY', 'B005EVCP1C', 'B0...",{},"<img src=""https://images-na.ssl-images-amazon....",NaN,"<div class=""a-fixed-left-grid a-spacing-none"">...",$32.89 - $35.24,B006N7Z32Y
86928,"['Clothing, Shoes & Jewelry', 'Men', 'Shoes', ...",NaN,We redefined classic oxford silhouettes: Rockp...,"class=""a-normal a-align-center a-spacing-smal...",Rockport Men's Sharp and Ready Colben Oxford,"['B00B1MA3MQ', 'B01M0OOJR6', 'B073HJG3VS', 'B0...",['https://images-na.ssl-images-amazon.com/imag...,NaN,NaN,"['100% Leather', 'Imported', 'Synthetic sole',...","10,835 in Clothing, Shoes & Jewelry (","['B01M0OOJR6', 'B00B1MA3MQ', 'B007UKYZHW', 'B0...",{},"<img src=""https://images-na.ssl-images-amazon....",NaN,"<div class=""a-fixed-left-grid a-spacing-none"">...",$49.99 - $143.00,B00EJTFRME


In [3]:
sample_desc = eval(metadata.description.tolist()[0])
sample_desc[0]

'The iconic, timeless chuck taylor all star sneaker. Perfect in its simplicity since 1917. Lightweight, breathable canvas construction. Vulcanized rubber sole delivers durable traction. An ortholite insole cushions each and every step. AND no one can ignore the unmistakable ankle patch. Versatile, fashionable, and they keep looking better every time you wear them. Ortholite is a trademark of O2 partners, llc.'

In [4]:
def text_preprocessing(text: str):
    
    # Lowercase
    text = text.lower()
    # Remove Punctuation
    # text = "".join([char for char in text if char not in string.punctuation])
    words = word_tokenize(text)  
    # Remove Stopwords
    stop_words = stopwords.words('english')
    filtered_words = [word for word in words if word not in stop_words]
    ## Stemming
    # porter = PorterStemmer()
    # stemmed = [porter.stem(word) for word in filtered_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in filtered_words]
    
    return ' '.join(lemmatized)

In [5]:
text_preprocessing(sample_desc[0])

'iconic , timeless chuck taylor star sneaker . perfect simplicity since 1917. lightweight , breathable canvas construction . vulcanized rubber sole delivers durable traction . ortholite insole cushion every step . one ignore unmistakable ankle patch . versatile , fashionable , keep looking better every time wear . ortholite trademark o2 partner , llc .'

In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')



d:\Softwares\Installed\Anaconda\envs\isr_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
list(metadata.description)

['The iconic, timeless chuck taylor all star sneaker. Perfect in its simplicity since 1917. Lightweight, breathable canvas construction. Vulcanized rubber sole delivers durable traction. An ortholite insole cushions each and every step. AND no one can ignore the unmistakable ankle patch. Versatile, fashionable, and they keep looking better every time you wear them. Ortholite is a trademark of O2 partners, llc.',
 'Move worry-free with this wireless sports bra from Enell. Made from high quality Nylon and LYCRA, this sports bra prevents skin chafing while providing optimum support. The front hook closure helps you wear or take off the bra easily. Contructed with Naturexx Performance Fabric that wicks away moisture to keep you cool and odor-free during the most intense workouts',
 'The Ctas Pro Ox is a fresh take on a Converse All Star..',
 "It's HOT! HOT! HOT! The most official looking bunker gear for kids anywhere. The Jr. Firefighter suit by Aeromax is sure to make little kids into big

In [23]:
encoded_data = model.encode(metadata.description.tolist(), device='cuda') 
encoded_data = np.asarray(encoded_data.astype('float32'))


In [24]:
encoded_title_data = model.encode(metadata.title.tolist(), device='cuda') 
encoded_title_data = np.asarray(encoded_title_data.astype('float32'))


In [10]:
with open('encodings_description.pickle','wb') as file:
    pickle.dump(encoded_data, file, protocol=pickle.HIGHEST_PROTOCOL)

with open('encodings_title.pickle','wb') as file:
    pickle.dump(encoded_title_data, file, protocol=pickle.HIGHEST_PROTOCOL)

with open('model.pickle','wb') as file:
    pickle.dump(model, file, protocol=pickle.HIGHEST_PROTOCOL)

In [44]:
with open('encodings.pickle','rb') as file:
    encoding_loaded = pickle.load(file)

with open('model.pickle','rb') as file:
    model_loaded = pickle.load(file)

In [27]:
def find_similar_items(query: str, model: SentenceTransformer, encodings: np.array, top_n: int = 5):
    query_vector = model.encode([query])
    # print(query_vector)
    similarity = np.dot(encodings,query_vector.T)
    # print(similarity)
    top_items = similarity.flatten().argsort()[-top_n:][::-1]
    # print(top_items)
    # return metadata['asin'].iloc[top_items]
    return top_items

In [32]:
top_n = 10
query = "women's earring"

top_items = find_similar_items(query=query, model=model, encodings=encoded_title_data, top_n=top_n)
metadata['title'].iloc[top_items]

56104    Yoursfs Ear Cuff Non-piercing Cuff Earring Hel...
24916    MG Jewelry Fashion Women's Stainless Steel Rou...
8761     14k Gold Plated Simulated Diamond Cz Heart Que...
69134    Lux Accessories Women's Girl's Assorted Multip...
81824    Anne Klein &quot;Classics&quot; Gold-Tone Band...
82892    Original Tribe Vintage Style Antique Bronze De...
52581    Necklace and Earring Bust Jewelry Display - White
14997    Women's Stainless Steel Clear Princess Cubic Z...
15169    FUNOC Womens Luxury Faux Fur Headband Winter W...
23403    Anna Belen &quot;Felina&quot; Glitter Cat Ears...
Name: title, dtype: object

In [33]:
top_n = 10
query = "men socks"

top_items = find_similar_items(query=query, model=model, encodings=encoded_title_data, top_n=top_n)
metadata['title'].iloc[top_items]

30215             100% Pure Wool Socks - Men, Natural Gray
23780    Gilbin Mens Winter Cozy Thick Warm Fuzzy Anti-...
56665    Foot Traffic - Men's Education-Themed Socks, F...
95992    Dickies Men's Multi-Pack Dri-tech Moisture Con...
92603    1 Pair Men's ExtraFine Merino Fun Dots OTC Ita...
6088     Gold Toe Men's Cotton Crew Athletic Sock 6-pac...
3348     Gold Toe Men&rsquo;s 6-Pack Cotton Quarter 656...
7052     Gold Toe Men&rsquo;s 6-Pack Cotton Quarter 656...
7050     Gold Toe Men's 6-Pack Cotton No Show 656 Athle...
3346     Gold Toe Men's 6-Pack Cotton No Show 656 Athle...
Name: title, dtype: object